In [ ]:
using HerbGrammar, HerbData, HerbSearch, HerbInterpret, HerbBenchmarks

import HerbInterpret.interpret

In [ ]:
function isnotuppercase(x::Char)
    return !isuppercase(x)
end

g = @cfgrammar begin
    # Booleans
    Bool = AtEnd # .==(Index, length(String)) # At end
    Bool = NotAtEnd # .<(Index, length(String)) # Not at end
    Bool = AtStart # .==(Number, 0) # At start
    Bool = NotAtStart # .>(Number, 0) # Not at start
    Bool = isletter
    Bool = x -> !isletter(x)
    Bool = isuppercase
    Bool = isnotuppercase
    Bool = islowercase
    Bool = x -> !islowercase(x)
    Bool = isnumeric
    Bool = x -> !isnumeric(x)
    Bool = isspace

    # Transitions
    Transition = left | right | uppercase | lowercase | drop

    # Not complete from here onwards...
    S = Transition; S | S; S | ϵ
    S = if Bool 
        S
    else
        S
    end
end

function interpret(tab::SymbolTable, ex::Expr)
    """
    Attempt at an interpreter, not complete.
    """
    args = ex.args
    input = tab[:Input]
    index = tab[:Pointer]

    if ex.head == :call
        if ex.args[1] == Symbol("left")
            tab[:Pointer] = max(1, index - 1)
            return interpret(tab, args[2])
        elseif ex.args[1] == Symbol("right")
            tab[:Pointer] = min(length(input) + 1, index + 1)
            return interpret(tab, args[2])
        elseif ex.args[1] == Symbol("uppercase")
            input_array = collect(input)
            input_array[index] = uppercase(input_array[index])
            tab[:Input] = join(input_array)
            if length(args) == 2  # Uppercase with further instructions
                return interpret(tab, args[2])
            else
                return tab[:Input]  # Uppercase without further instructions
            end
        elseif ex.args[1] == Symbol("lowercase")
            input_array = collect(input)
            input_array[index] = lowercase(input_array[index])
            tab[:Input] = join(input_array)
            return interpret(tab, args[2])
        elseif ex.args[1] == Symbol("drop")
            println("attempting to drop at $index")
            tab[:Input] = join(deleteat!(collect(input), index))
            tab[:Pointer] = min(index, length(tab[:Input]))
            if length(args) == 2  # Drop with further instructions
                return interpret(tab, args[2])
            else
                return tab[:Input]  # Drop without further instructions
            end
        else
            if length(args) == 1
                return tab[args[1]]()
            elseif length(args) == 2
                tab[args[1]](interpret(tab, args[2]))
            else
                error("Function not supported")
            end
        end
    elseif ex.head == :while
        # TODO: implement while loop
    elseif ex.head == :if
        println("if statement")
        if interpret(tab, args[1])
            return interpret(tab, args[2])
        else
            return interpret(tab, args[3])
        end
    else
        error("Expression type not supported")
    end
end

tab = SymbolTable(g)
input = Dict(:Pointer => 1, :Input => "abd")
merge(tab, input)
ex = :(right(right(uppercase())))

interpret(input, ex)